In [15]:
# First install everything -- you only need to do this once: 
#! pip install https://ray-ci-artifact-pr-public.s3.amazonaws.com/c58874ae8545eef0b5c7632418eba3da0b5015c9/tmp/artifacts/.whl/ray-3.0.0.dev0-cp38-cp38-manylinux2014_x86_64.whl
#! pip install ray[tune]
#! pip install torch
#! pip install torchvision
#! pip install tqdm
# ! pip install anyscale
# %env ANYSCALE_HOST=https://console.anyscale-staging.com
# %env ANYSCALE_CLI_TOKEN=sss_4nneyStJk8ORsSgxW45lDT
# %env IGNORE_VERSION_CHECK=1


env: ANYSCALE_HOST=https://console.anyscale-staging.com
env: ANYSCALE_CLI_TOKEN=sss_4nneyStJk8ORsSgxW45lDT
env: IGNORE_VERSION_CHECK=1


In [16]:
FILES_DIR = "s3://waleed-movies"

# files_dir = "/home/ec2-user/images"

# Use this for quick testing with ~300 files
# FILES_DIR = "s3://air-example-data-2/movie-image-small-filesize-1-file"
import numpy as np
import json

import ray
from ray.train.torch import TorchPredictor, TorchCheckpoint
from ray.train.batch_predictor import BatchPredictor
from ray.data.preprocessors import BatchMapper
from ray.data.datasource import ImageFolderDatasource
import anyscale

from torchvision.models.detection.ssd import ssd300_vgg16

from util import visualize_objects, convert_to_tensor, SSDPredictor

# TODO: Enable auto casting once we resolve call_model() output format
from ray.data.context import DatasetContext
ctx = DatasetContext.get_current();
ctx.enable_tensor_extension_casting = False

In [17]:
def batch_predict(files_dir, lim):
    dataset = ray.data.read_datasource(
        ImageFolderDatasource(), root=files_dir, size=(300, 300), mode="RGB"
    ).limit(lim)

    preprocessor = BatchMapper(convert_to_tensor)
    model = ssd300_vgg16(pretrained=True)
    ckpt = TorchCheckpoint.from_model(model=model, preprocessor=preprocessor)
    predictor = BatchPredictor.from_checkpoint(ckpt, SSDPredictor)
    return predictor.predict(dataset, 
                             batch_size=128,
                             min_scoring_workers=20,
                             max_scoring_workers=20, 
                             num_cpus_per_worker=4, 
                             num_gpus_per_worker=1, 
                             feature_columns=["image"], 
                             keep_columns=["image"])

(scheduler +16m29s) Resized to 48 CPUs, 4 GPUs.
(scheduler +16m29s) Resized to 240 CPUs, 20 GPUs.
(scheduler +16m39s) Resized to 48 CPUs, 4 GPUs.
(scheduler +16m39s) Resized to 240 CPUs, 20 GPUs.
(scheduler +16m48s) Resized to 48 CPUs, 4 GPUs.
(scheduler +16m48s) Resized to 240 CPUs, 20 GPUs.
(scheduler +16m58s) Resized to 48 CPUs, 4 GPUs.
(scheduler +16m58s) Resized to 240 CPUs, 20 GPUs.
(scheduler +17m9s) Resized to 48 CPUs, 4 GPUs.
(scheduler +17m9s) Resized to 240 CPUs, 20 GPUs.
(scheduler +17m18s) Resized to 48 CPUs, 4 GPUs.
(scheduler +17m18s) Resized to 240 CPUs, 20 GPUs.
(scheduler +17m27s) Resized to 48 CPUs, 4 GPUs.
(scheduler +17m27s) Resized to 240 CPUs, 20 GPUs.
(scheduler +17m37s) Resized to 48 CPUs, 4 GPUs.
(scheduler +17m37s) Resized to 240 CPUs, 20 GPUs.
(scheduler +17m46s) Resized to 48 CPUs, 4 GPUs.
(scheduler +17m46s) Resized to 240 CPUs, 20 GPUs.
(scheduler +17m56s) Resized to 48 CPUs, 4 GPUs.
(scheduler +17m56s) Resized to 240 CPUs, 20 GPUs.
(scheduler +18m5s) Res

In [8]:
ray.init("anyscale://workspace-project-demo/workspace-cluster-demo", runtime_env={"working_dir": "."})

Parsing Ray Client arguments
Finished parsing arguments.

Choosing a project
Using the project workspace-project-demo:
  name:               workspace-project-demo
  project id:         prj_cNsZAtGnE6FU5tczBLqVsJv3

Preparing the cluster
Cluster workspace-cluster-demo is currently running.
Connecting to this cluster:
  cluster id:                   ses_Lz9BqdCTSjv99SNRMjPKwTRv
  cluster environment:          apt_9zd5xA9LDSNTibWFSgXmCTkx:3
  cluster environment id:       bld_LuYuxCsYgZG1e2afMLQx9YxE
  cluster compute:              demo_cluster_compute_3a048671-4976-4960-9d9b-f931899f51e1
  cluster compute id:           cpt_CHAYTmE2MNzjuN89Zsxx7rWL
  idle termination:             120 minutes
  maximum uptime:               disabled
  link:                         https://console.anyscale-staging.com/projects/prj_cNsZAtGnE6FU5tczBLqVsJv3/clusters/ses_Lz9BqdCTSjv99SNRMjPKwTRv

Starting the interactive session
Uploading `working_dir: .` to the cluster ...


(scheduler +1s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +1s) Resized to 48 CPUs, 4 GPUs.


Connected to workspace-cluster-demo.
[Warning] Unable to print information for interactive session with job name ipykernel_launcher.py_08-03-2022_21:06:28. Please view cluster at https://console.anyscale-staging.com/projects/prj_cNsZAtGnE6FU5tczBLqVsJv3/clusters/ses_Lz9BqdCTSjv99SNRMjPKwTRv.


Python version:,3.8.5
Ray version:,3.0.0.dev0
Dashboard:,http://https://session-lz9bqdctsjv99snrmjpkwtrv.i.anyscaleuserdata-staging.com/auth/?token=agh0_CkcwRQIhANi21ZCVoKkIHTGnFFtfysDcoLRRO8Z1SKJNuziVXXF6AiAte1L7o79zvq3kjX5iZXZ0KDWwraW-GqY1ACOMcQTj4hJmEiAlBPw5uCiEeqKg553UhWeGNgxNvc_2tBVrwvBWh8uXyhgCIgNuL2E6DAjVyZXiBhDY9p-rA0IMCNXG_ZYGENj2n6sD-gEeChxzZXNfTHo5QnFkQ1RTanY5OVNOUk1qUEt3VFJ2&redirect_to=dashboard


(scheduler +10s) Resized to 48 CPUs, 4 GPUs.
(scheduler +18s) Resized to 48 CPUs, 4 GPUs.
(scheduler +27s) Resized to 48 CPUs, 4 GPUs.
(scheduler +38s) Resized to 48 CPUs, 4 GPUs.


In [11]:
prediction_results = batch_predict(FILES_DIR, 100000)


(scheduler +6m42s) Resized to 48 CPUs, 4 GPUs.
(scheduler +6m42s) Resized to 240 CPUs, 20 GPUs.
(scheduler +6m50s) Resized to 48 CPUs, 4 GPUs.
(scheduler +6m50s) Resized to 240 CPUs, 20 GPUs.


(_map_block_nosplit pid=1046, ip=172.31.81.117) E0803 14:13:26.212298576   12992 chttp2_transport.cc:1103]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(_map_block_nosplit pid=1054, ip=172.31.81.117) E0803 14:13:32.045143373   13003 chttp2_transport.cc:1103]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
Read progress:   9%|▉         | 42/480 [00:02<00:23, 18.31it/s]

(scheduler +7m0s) Resized to 48 CPUs, 4 GPUs.
(scheduler +7m0s) Resized to 240 CPUs, 20 GPUs.


Read progress:  46%|████▌     | 220/480 [00:12<00:13, 18.80it/s]

(scheduler +7m10s) Resized to 48 CPUs, 4 GPUs.
(scheduler +7m10s) Resized to 240 CPUs, 20 GPUs.


...

(scheduler +9m17s) Resized to 48 CPUs, 4 GPUs.
(scheduler +9m17s)

Map Progress (20 actors 0 pending):  88%|████████▊ | 422/480 [00:59<00:07,  7.54it/s]

 Resized to 240 CPUs, 20 GPUs.


Map Progress (20 actors 0 pending): 100%|██████████| 480/480 [01:06<00:00,  7.24it/s]


(scheduler +9m25s) Resized to 48 CPUs, 4 GPUs.
(scheduler +9m25s) Resized to 240 CPUs, 20 GPUs.
(scheduler +9m35s) Resized to 48 CPUs, 4 GPUs.
(scheduler +9m35s) Resized to 240 CPUs, 20 GPUs.


In [12]:
prediction_results

Dataset(num_blocks=480, num_rows=41979, schema={boxes: object, scores: object, labels: object, image: object})

(scheduler +9m46s) Resized to 48 CPUs, 4 GPUs.
(scheduler +9m46s) Resized to 240 CPUs, 20 GPUs.


TypeError: 'Dataset' object is not subscriptable

(scheduler +10m31s) Resized to 48 CPUs, 4 GPUs.
(scheduler +10m31s) Resized to 240 CPUs, 20 GPUs.
(scheduler +10m41s) Resized to 48 CPUs, 4 GPUs.
(scheduler +10m41s) Resized to 240 CPUs, 20 GPUs.
(scheduler +10m52s) Resized to 48 CPUs, 4 GPUs.
(scheduler +10m52s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m1s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m1s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m12s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m12s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m22s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m22s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m31s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m31s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m40s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m40s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m50s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m50s) Resized to 240 CPUs, 20 GPUs.
(scheduler +11m59s) Resized to 48 CPUs, 4 GPUs.
(scheduler +11m59s) Resized to 240 CPUs, 20 GPUs.
(scheduler +12m9s) Res